##    <center> DSCI 100 Group Project Report
    
**Group 168**

**Anjali Dajee**

**Ross Tomita**

**Yuwen Luo**<center>
    
    
______________

# <center>Investigating pulsar star data to find an accurate predictive model using the KNN algorithm: exploratory study <center>

In [7]:
# 1. Introduction

Pulsars are neutron stars that form when a star collapses within itself and eventually explodes, creating a supernova. As the pulsars rotate, they produce a broadband radio emission that can be detected from earth using a large dish antenna and a radio. Since each pulsar’s emission is slightly different during each rotation, a singular signal detection is made by averaging several rotation emissions of the star (Lyon et al.)  However, most detections are actually interfering man-made radio signals, and not actual pulsars. To determine whether a detection is truly a pulsar, other variables must be considered. The discovery of pulsars has allowed scientists to observe neutron stars for the first time, as well as test some of the conditions of Einstein’s theory of relativity.

In the current study, we will explore the HTRU2 dataset which describes samples of known pulsar and non-pulsar stars. This dataset is described by 9 different attributes in which 8 quantitative variables come from two different curves:

- The first is an `average integrated pulse profile` of a single pulsar star. To account for minor discrepancies in each pulse, several emissions are averaged and put together to create a single pulse profile for a star. 

- The second curve accounts for delay when pulses arrive at differing time across different radio frequencies. This delay, or dispersion, can be fit and compensated for in astronomers’ calculations, however there is always a certain measure of uncertainty. This is accounted for in the “dispersion-measure-signal-to-noise-ratio” `(DM-SNR) curve`. 

From these two curves, 8 numerical features can be calculated and standardized. From the integrated pulse profile, we get;  `Mean, Excess Kurtosis, Skewness, and Standard Deviation`. From the DM-SNR curve, the `same four variables` can be acquired. 

The ninth variable, `Class`, is a categorical variable with 2 different levels, which is what we will be trying to predict (Scaife). 
<blockquote>
    0 represents negative examples (false detections for pulsar starts);
</blockquote>
<blockquote>
    1 represents positive examples (true detections for pulsar starts).
</blockquote>

We will use these in our analysis to determine **whether any of these can be used to accurately classify whether a detection is a pulsar star or not**. We have also taken into consideration that the number of true pulsar stars is a minority positive class, while the number of false detections is a majority negative class. This information will be useful when sampling  the training set from the original dataset. 


_____________________________________

In [ ]:
# 2. Methods & Results

In [ ]:
## 2.1 Import data, tidy data

## 2.2 Split data into training and testing sets 

In [ ]:
### 2.3 Pre-processing the training set

In [ ]:
### 2.3.1 Resolve imbalance problem by sampling down the 0 (negative) class

In [ ]:
### 2.3.2 Scale the training set after sampling down it

In [ ]:
### 2.3.3 Visualize the training set

In [ ]:
## 2.4 Data Analysis: K-NN Classification 

In [ ]:
## 2.4.1 Classification: using predictors chosen from Forward selection

In [ ]:
## 2.4.2 Classification: using predictors chosen from Correlation Matrix

In [ ]:
# 3. Multicollinearity Problem and Additional Analysis

In [ ]:
## 3.1  Multicollinearity Problem

In [ ]:
## 3.2 Additional Analysis with PCA (Principal Component Analysis)

________________________________

In [4]:
# 4. Discussion

In [6]:
## 4.1 Conclusions and Potential Limitations for Improvement

## 4.2 Impact and Future Questions

The question of whether the 96% accuracy is high enough to use in the real world depends on what this data is being used for. For general interest and scientific exploration, it’s a fairly good number and holds a high certainty. However, if this data were to be used for projects involving human space exploration, for example, or even simply in calculations involving space projects, that 4% uncertainty can pose a big risk to safety. Despite the accuracy being an objectively high percent, there are some instances where even a small amount of risk can be deadly. However, pulsar star data is generally used to test mathematical conditions, and as such, the accuracy is sufficient.

This data analysis could help promote the automation of pulsar classification, which would save astronomers a lot of time and effort. At present, each data point must be manually analyzed, and seeing as there’s upwards of 10,000 entries, this task is very time consuming. Using a model to predict the classification of pulsar stars would allow scientists to spend more time applying this data to further research. 

Our classification can lead to future questions, such as whether we can further classify stars using the same model. For example, differentiating between pulsars, stars, white dwarfs, black holes, and more. With more resources and knowledge of classification, this is a potential question we can one day explore.

________________________________

# WORKS CITED

R. J. Lyon, B. W. Stappers, S. Cooper, J. M. Brooke, J. D. Knowles, Fifty Years of Pulsar Candidate 
Selection: From simple filters to a new principled real-time classification approach, Monthly Notices of the Royal Astronomical Society 459 (1), 1104-1123, DOI: 10.1093/mnras/stw656

R. J. Lyon, HTRU2, DOI: 10.6084/m9.figshare.3080389.v1.

Scaife, A. (n.d.). CSC2019 - Introduction to Machine Learning. CSC2019 - Introduction to machine learning. Retrieved December 4, 2022, from https://as595.github.io/classification/ 